In [2]:
#import libraries
import boto3
import json
import time

In [3]:
# Connection to AWS
folder = '/home/propietari/Documents/claus/' # TODO: change to your local path
file_name = 'AWS_S3_keys_JordiPlanas_Made_in_game.json' # TODO: Change to your filename
folder = 'C:/Users/jordi/Documents/claus/' # TODO: change to your local path
file_name = 'AWS_S3_keys_wri.json' # TODO: Change to your filename
file = folder + file_name

with open(file, 'r') as dict:
    credentials = json.load(dict)
                                      
KEY = list(credentials)[0]
SECRET = list(credentials.values())[0]
# s3BucketName = "wri-testing"
s3BucketName = "wri-nlp-policy"
# region = 'eu-central-1'
region = "us-east-1"

s3 = boto3.resource(
    service_name = 's3',
    region_name = region,
    aws_access_key_id = KEY,
    aws_secret_access_key = SECRET
)

In [8]:
# prefix = "english_documents/raw_pdf/"
# i = 0
# for object in s3.Bucket(s3BucketName).objects.all().filter(Prefix=prefix):
#     srcKey = object.key
#     if ".txt" in srcKey:
#         print(i)
#         newKey = srcKey.replace(".txt", ".pdf")
#         copySource = s3BucketName + '/' + srcKey         
#         s3.Object(s3BucketName, newKey).copy_from(CopySource=copySource)
#         s3.Object(s3BucketName, srcKey).delete()
#     i += 1

3
5
6
8
9
11
12
13
14
15
17
20
22
23
25
26
27
28
29
30
31
38
39
41
42
43
44
45
46
47
48
49
50
54
57
58
59
61
62
63
69
73
75
76
77
80
81
82
83
84
85
86
87
88
89
95
97
98
101
103
104
106
107
108
110
111
112
113
114
115
116
118
119
120
123
124
126
128
131
135
136
138
139
140
142
143
144
146
150
151
155
163
165
166
168
170
171
174
175
180
181
184
185
186
188
189
191
192
194
195
197
198
199
201
202
203
204
205
206
209
210
213
217
221
222
224
228
229
231
236
237
239
240
243
244
245
246
247
250
251
252
256
257
260
266
267
268
269
270
271
273
274
276
279
280
281
283
284
285
286
291
292
294
300
301
302
304
306
307
309
312
313
315
316
319
320
322
323
324
325
326
327
328
331
332
333
335
336
337
339
340
341
342
345
346
347
352
354
356
358
359
361
363
368
369
370
373
374
376
377
378
379
380
382
386
387
390
393
399
400
404
405
406
407
408
409
410
411
412
413
418
419
420
421
422
425
426
427
429
431
432
434
435
438
439
440
442
443
444
449
453
455
456
457
458
460
464
469
470
472
473
474
475
476
477
480

In [4]:
def startJob(s3BucketName, objectName):
    response = None
    client = boto3.client('textract',
#                           service_name = 's3',
                          region_name = region,
                          aws_access_key_id = KEY,
                          aws_secret_access_key = SECRET)
    response = client.start_document_text_detection(DocumentLocation={
        'S3Object': {'Bucket': s3BucketName,
                     'Name': objectName
                    }
        })
    return response["JobId"]
def isJobComplete(jobId):
 # For production use cases, use SNS based notification 
 # Details at: https://docs.aws.amazon.com/textract/latest/dg/api-async.html
    time.sleep(5)
    client = boto3.client('textract',
#                           service_name = 's3',
                          region_name = region,
                          aws_access_key_id = KEY,
                          aws_secret_access_key = SECRET)
    response = client.get_document_text_detection(JobId=jobId)
    status = response["JobStatus"]
#     print("Job status: {}".format(status))
    while(status == "IN_PROGRESS"):
        time.sleep(5)
        response = client.get_document_text_detection(JobId=jobId)
        status = response["JobStatus"]
#         print("Job status: {}".format(status))
    return status
def getJobResults(jobId):
    pages = []
    client = boto3.client('textract',
#                           service_name = 's3',
                          region_name = region,
                          aws_access_key_id = KEY,
                          aws_secret_access_key = SECRET)
    response = client.get_document_text_detection(JobId=jobId)
    
    pages.append(response)
#     print("Resultset page recieved: {}".format(len(pages)))
    nextToken = None
    if('NextToken' in response):
        nextToken = response['NextToken']
        while(nextToken):
            response = client.get_document_text_detection(JobId=jobId, NextToken=nextToken)
            pages.append(response)
#             print("Resultset page recieved: {}".format(len(pages)))
            nextToken = None
            if('NextToken' in response):
                nextToken = response['NextToken']
    return pages

def save_txt_file(file, file_name):
    f = open(file_name, "w")
    f.write(file)
    f.close()

In [5]:
def AWS_pdf_to_text(key, s3BucketName):
    # Document
    documentName = key
    jobId = startJob(s3BucketName, documentName)
#     print("Started job with id: {}".format(jobId))
    if(isJobComplete(jobId)):
        response = getJobResults(jobId)
    #print(response)
    # Print detected text
    policy = ""
    for resultPage in response:
#         print(resultPage)
        try:
            for item in resultPage["Blocks"]:
                if item["BlockType"] == "LINE":
                    policy = policy + item["Text"] + "\n"
#                     print ('\033[94m' + item["Text"] + '\033[0m')
        except:
            f = open("../output/log_file.txt", 'a')
            f.write(f"The file {key} could not be converted\n")
            f.close()
            
            print(f"The file {key} could not be converted")
    return policy

In [6]:
keyword = "0bc17448527c7798e84937eb897bdaa82c3c4a3c.pdf"
prefix = "english_documents/raw_pdf/"
i = 0


for obj in s3.Bucket(s3BucketName).objects.all().filter(Prefix = prefix):#.:
    if ".pdf" in obj.key and 400 > i > 199: #lower_limit < i < upper_limit:
        print(i, "**", obj.key)
        key = obj.key.replace("raw_pdf", "text_files").replace(".pdf", ".txt")
        policy = AWS_pdf_to_text(obj.key, s3BucketName)
        if policy != "":
            s3.Object(s3BucketName, key).put(Body = policy)
#         save_txt_file(policy, "/home/propietari/Documents/S3/" + obj.key[:-4] + ".txt")
    i += 1

200 ** english_documents/raw_pdf/43fd1c3c2b65e8a586a34995feb553be7de5ed8d.pdf
201 ** english_documents/raw_pdf/444cf6267e3db5e5361af741c1b2efadb75e2af8.pdf
202 ** english_documents/raw_pdf/449265a978ca85a21f17abe93c48109a107d616d.pdf
203 ** english_documents/raw_pdf/44981e34e82a8051151f95b01c8a9a4ec9d9bd03.pdf
204 ** english_documents/raw_pdf/44b37c21a860f8eeedc53977e53074b326f19ca4.pdf
205 ** english_documents/raw_pdf/4524299345c9df6978c9a79815af3b4e2592772c.pdf
206 ** english_documents/raw_pdf/4537c8e41c05c3f9c73f6f5813efb4b4da7bc09f.pdf
207 ** english_documents/raw_pdf/45d04a8fad93a718b4ce784ee0fa1f7ad6735313.pdf
208 ** english_documents/raw_pdf/45dbf2f7838c8e8cdeb04b792faa230d49a40d3e.pdf
209 ** english_documents/raw_pdf/45f5f381eaf58ca6e1cef032f026fbdb06f0d442.pdf
210 ** english_documents/raw_pdf/4646164a4a8ad36a183c270df53ec3cd31dde67c.pdf
211 ** english_documents/raw_pdf/46b264eb143d965b0ca986ef2f6dd2a1610d2ab7.pdf
212 ** english_documents/raw_pdf/46b2f6519c30f6505a490e7e06c6725

306 ** english_documents/raw_pdf/5c4e34319089eb966b322ea784c3f24261d63d59.pdf
307 ** english_documents/raw_pdf/5c5cb2d709be34fc71dba84029ebd64722d0ce7d.pdf
308 ** english_documents/raw_pdf/5d23a8189540ff9230f76260d8a64b8006841434.pdf
309 ** english_documents/raw_pdf/5d2b07082c077bdbb026e3b77f33d1db75162ed2.pdf
310 ** english_documents/raw_pdf/5db5cc44bad37822d1bd3e83bf1021d137df65e6.pdf
311 ** english_documents/raw_pdf/5dd383b4082ed86e9488a2994faba1cc0878c0cd.pdf
312 ** english_documents/raw_pdf/5e1458b168b144c9067b7ad67df2a395259192ff.pdf
313 ** english_documents/raw_pdf/5e412c1d748c2cc979056a57efcccc0ed02ed096.pdf
314 ** english_documents/raw_pdf/5edd39ad033ee5c5f4827ce1deaa25539ed8e117.pdf
315 ** english_documents/raw_pdf/5ef8d6c775ea1cdaac0645a64fef1b81323616b8.pdf
316 ** english_documents/raw_pdf/5f877a8d1eba4a84496ee9b51492050fd359f667.pdf
317 ** english_documents/raw_pdf/5fdd872bd380370332436d347a560ec37f60142b.pdf
318 ** english_documents/raw_pdf/5ff9cee3566ec61b3f51ee2273a7bb1

In [ ]:
policy